# Read Data

In [1]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('SUI_USDT_futures_15m_20241231.csv')

# Drop unused columns
df.drop(columns=['coin','unused', 'closetime'], inplace=True)

# Drop unused columns
df.drop(columns=['volume','quotevolume', 'trades', 'taker_buy_volume', 'taker_buy_quote'], inplace=True)

# Display the first few rows of the DataFrame
df

,opentime,openprice,highprice,lowprice,closeprice
0,1683129600000,1.4030,1.4030,1.2652,1.3338
1,1683130500000,1.3339,1.3863,1.3327,1.3716
2,1683131400000,1.3717,1.3957,1.3547,1.3857
3,1683132300000,1.3866,1.3924,1.3652,1.3733
4,1683133200000,1.3731,1.3756,1.3405,1.3574
...,...,...,...,...,...
58323,1735620300000,4.0787,4.0850,4.0616,4.0828
58324,1735621200000,4.0829,4.0953,4.0738,4.0810
58325,1735622100000,4.0811,4.0990,4.0637,4.0982
58326,1735623000000,4.0982,4.1310,4.0900,4.1198


# Explore

In [9]:
df_setup = df.copy()

# Calculate candle body size
df_setup['body_size'] = abs(df_setup['closeprice'] - df_setup['openprice'])

# Calculate tail length
df_setup['high_tail'] = abs(df_setup['highprice'] - df_setup[['openprice', 'closeprice']].max(axis=1))
df_setup['low_tail'] = abs(df_setup['lowprice'] - df_setup[['openprice', 'closeprice']].min(axis=1))

# Calculate tail to body ratio
df_setup['high_tail_ratio'] = df_setup['high_tail'] / df_setup['body_size']
df_setup['low_tail_ratio'] = df_setup['low_tail'] / df_setup['body_size']

# Calculate body + tail
df_setup['body_high_tail_ratio'] = (df_setup['high_tail'] + df_setup['body_size']) / df_setup['openprice'] * 100
df_setup['body_low_tail_ratio'] = (df_setup['low_tail'] + df_setup['body_size']) / df_setup['openprice'] * 100

#Get next openprice
df_setup['next_openprice'] = df_setup['openprice'].shift(-1)

# Add a new column 'next_trend' and set default value to 0
df_setup['next_trend'] = 0
df_setup.loc[(df_setup['high_tail_ratio'] >= 1) & (df_setup['body_high_tail_ratio'] >= 0.5), 'next_trend'] = -1
df_setup.loc[(df_setup['low_tail_ratio'] >= 1) & (df_setup['body_low_tail_ratio'] >= 0.5), 'next_trend'] = 1

df_setup.dropna(inplace=True)

df_setup

,opentime,openprice,highprice,lowprice,closeprice,body_size,high_tail,low_tail,high_tail_ratio,low_tail_ratio,body_high_tail_ratio,body_low_tail_ratio,next_openprice,next_trend
0,1683129600000,1.4030,1.4030,1.2652,1.3338,0.0692,0.0000,0.0686,0.000000,0.991329,4.932288,9.821810,1.3339,0
1,1683130500000,1.3339,1.3863,1.3327,1.3716,0.0377,0.0147,0.0012,0.389920,0.031830,3.928330,2.916261,1.3717,0
2,1683131400000,1.3717,1.3957,1.3547,1.3857,0.0140,0.0100,0.0170,0.714286,1.214286,1.749654,2.259969,1.3866,1
3,1683132300000,1.3866,1.3924,1.3652,1.3733,0.0133,0.0058,0.0081,0.436090,0.609023,1.377470,1.543343,1.3731,0
4,1683133200000,1.3731,1.3756,1.3405,1.3574,0.0157,0.0025,0.0169,0.159236,1.076433,1.325468,2.374190,1.3570,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58322,1735619400000,4.1052,4.1115,4.0773,4.0787,0.0265,0.0063,0.0014,0.237736,0.052830,0.798987,0.679626,4.0787,0
58323,1735620300000,4.0787,4.0850,4.0616,4.0828,0.0041,0.0022,0.0171,0.536585,4.170732,0.154461,0.519773,4.0829,1
58324,1735621200000,4.0829,4.0953,4.0738,4.0810,0.0019,0.0124,0.0072,6.526316,3.789474,0.350241,0.222881,4.0811,0
58325,1735622100000,4.0811,4.0990,4.0637,4.0982,0.0171,0.0008,0.0174,0.046784,1.017544,0.438607,0.845360,4.0982,1


In [11]:
# Initialize variables
indices = []  # To store the captured indices and alternating trends
previous_trend = None  # To track the last trend (`1` or `-1`)
previous_index = None  # To track the last index of the trend

# Iterate through the DataFrame
for idx, row in df_setup.iterrows():
    current_trend = row['next_trend']  # Current value of next_trend

    # Check if it's the first trend (start tracking)
    if previous_trend is None:
        if current_trend in [1, -1]:
            previous_trend = current_trend
            previous_index = idx
    else:
        # Look for alternating trends
        if previous_trend == 1 and current_trend == -1:
            indices.append((previous_trend, previous_index, current_trend, idx))
            previous_trend = -1
            previous_index = idx
        elif previous_trend == -1 and current_trend == 1:
            indices.append((previous_trend, previous_index, current_trend, idx))
            previous_trend = 1
            previous_index = idx

# Convert the captured indices into a DataFrame
result_df = pd.DataFrame(indices, columns=['Start_Trend', 'Start_Index', 'End_Trend', 'End_Index'])

# Filter df_setup based on Start_Index from result_df
filtered_df_setup = df_setup.loc[result_df['Start_Index']]

# Reset the index for better readability (optional)
filtered_df_setup.reset_index(drop=True, inplace=True)

# Display the filtered DataFrame
filtered_df_setup

,opentime,openprice,highprice,lowprice,closeprice,body_size,high_tail,low_tail,high_tail_ratio,low_tail_ratio,body_high_tail_ratio,body_low_tail_ratio,next_openprice,next_trend
0,1683131400000,1.3717,1.3957,1.3547,1.3857,0.0140,0.0100,0.0170,0.714286,1.214286,1.749654,2.259969,1.3866,1
1,1683155700000,1.3787,1.4016,1.3771,1.3856,0.0069,0.0160,0.0016,2.318841,0.231884,1.660985,0.616523,1.3858,-1
2,1683156600000,1.3858,1.4125,1.3656,1.3773,0.0085,0.0267,0.0117,3.141176,1.376471,2.540049,1.457642,1.3773,1
3,1683162000000,1.4652,1.4819,1.4611,1.4670,0.0018,0.0149,0.0041,8.277778,2.277778,1.139776,0.402675,1.4670,-1
4,1683165600000,1.4477,1.4525,1.4290,1.4401,0.0076,0.0048,0.0111,0.631579,1.460526,0.856531,1.291704,1.4401,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6423,1735573500000,4.0088,4.0376,4.0042,4.0203,0.0115,0.0173,0.0046,1.504348,0.400000,0.718419,0.401616,4.0202,-1
6424,1735576200000,4.0524,4.0785,4.0343,4.0697,0.0173,0.0088,0.0181,0.508671,1.046243,0.644063,0.873556,4.0695,1
6425,1735584300000,4.2439,4.2782,4.2419,4.2587,0.0148,0.0195,0.0020,1.317568,0.135135,0.808219,0.395862,4.2586,-1
6426,1735590600000,4.2826,4.3032,4.2632,4.2849,0.0023,0.0183,0.0194,7.956522,8.434783,0.481016,0.506702,4.2849,1
